## 대용량 비정형 데이터 기반 Advanced RAG 구축 및 평가
#### 강의 목표
1. **Real-World Data Handling**: 나무위키(구어체/지식) FinePDFs(문어체/전문문서)가 섞인 두개 데이터를 처리합니다.
2. **Beyond Keyword Search**: 단순 키워드 매칭(BM25)과 의미 기반 검색(Vector Search)의 결정적 차이를 체감합니다.
3. **Hybrid Search**: BM25와 Vector Search를 결합하여 검색 성능을 향상시킵니다.
4. **Advanced Pipeline**: Retriever + Reranker 구조를 통해 검색 정확도를 극대화하는 기법을 익힙니다.
5. **Query Decomposition**: 복잡한 질의를 분해하여 더 정확한 검색 결과를 얻는 방법을 배웁니다.
6. **Data-Driven Evalution**: 정답지가 없는 상황에서 LLM을 이용해 *합성 데이터셋(Synthetic Testset)*을 만들고 평가를 수행합니다.

#### 사용 모델 및 데이터
- **LLM**: gpt-5-mini (High Performance & Low Cost)
- **Data**: heegyu/namuwiki (General Knowledge) + HuggingFaceFW/finepdfs-edu (Domain Documents)

In [ ]:
# 1. 필수 라이브러리 설치 (Jupyter 환경 권장 방식)

# 버전을 명시하여 호환성 문제 방지

%pip install -q \
    langchain==0.3.14 \
    langchain-openai==0.2.14 \
    langchain-huggingface==0.1.2 \
    langchain-community==0.3.14 \
    ragas==0.2.10 \
    datasets \
    sentence-transformers \
    tiktoken \
    rank_bm25 \
    pandas \
    faiss-cpu

In [ ]:
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# API_KEY 정보 로드
load_dotenv()


In [ ]:
# print API key
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY'][:-15]}" + "*" * 15)

## 1. Data Loading: '현실적인' 데이터 믹싱(Mixing)
현실에는 깔끔하게 정제된 데이터만 존재하지 않습니다. **지식 백과(Wiki)** 스타일과 **보고서(PDF)** 스타일의 텍스트가 섞여서, 난이도가 높은 통합 검색 환경을 구축합니다.

In [ ]:
from datasets import load_dataset
from langchain.docstore.document import Document

print("📥 데이터 스트리밍 시작...")

raw_docs = []

# [Source A] 나무위키 (General Knowledge)
# 특징: 구어체, 다양한 주제, 메타데이터에 제목 포함
wiki_stream = load_dataset("heegyu/namuwiki", split="train", streaming=True)
for idx, data in enumerate(wiki_stream):
    if idx >= 10: break  # 실습 속도를 위해 100개만 사용, Memory & Computation 여유가 있다면 전부 써보기
    # print(f"[{idx+1}/100] {data}")
    content = f"[출처: 나무위키] 제목: {data['title']}\n\n{data['text']}"
    raw_docs.append(Document(page_content=content, metadata={"source": "namuwiki", "title": data['title']}))

for i, doc in enumerate(raw_docs):
    print(f"[{i+1}/10] {doc}")

